下面的代码是尝试构建两个较小的向量数据库复现问题的代码，具体来说：

1. 将问题「ICI House is now named after the company that provides what type of item?」得到的检索结果存入 langchain 的 Document 对象中

2. 按照之前相同的方法构建了 2 个类似的小向量数据库，并将 Document 对象利用 `add_documents` 函数存入其中。存入的实体数量为 `20`

3. 按照之前相同的方式在两个向量数据库上对问题进行密集检索

In [ ]:
# 将第一个 top_1 不同问题 两个向量数据库检索到的内容都输出出来
# 构建一个类似的向量数据库，看能否复现出来类似的问题
d_res = top1_diff[0]['dense_res']
h_res = top1_diff[0]['hybrid_res']

for d_r, h_r in zip(d_res, h_res):
    print('-' * 50)
    print(f'dense_score: \n{d_r[1]}\n')
    print(f'hybrid_score: \n{h_r[1]}\n')
    print(f'dense_content: \n{d_r[0].page_content}\n')
    print(f'hybrid_content: \n{h_r[0].page_content}\n')
    print()

--------------------------------------------------
dense_score: 
0.7522499561309814

hybrid_score: 
0.9679315090179443

dense_content: 
ICI House (now Orica House) is a 19-storey office building in East Melbourne, Victoria, Australia. Begun in 1955, it was the tallest building in Australia upon completion in 1958, breaking Melbourne's long standing 132ft height limit, and was the first International Style skyscraper in the country. It symbolised progress, modernity, efficiency and corporate power in postwar Melbourne, and heralded the construction of the high-rise office buildings

hybrid_content: 
, then a record among magazines published by Time, Inc., SI's parent company.


--------------------------------------------------
dense_score: 
0.8594906330108643

hybrid_score: 
0.9679315090179443

dense_content: 
ICI Fibres developed the Crimplene fibre. It is a thick, polyester yarn used to make a fabric of the same name. The resulting cloth is heavy, wrinkle-resistant and retains its sh

In [ ]:
# 定义嵌入模型
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

embed_model = OpenAIEmbeddings(
    api_key=os.getenv("BL_API_KEY"),
    base_url=os.getenv("BL_BASE_URL"),
    model="text-embedding-v3",
    dimensions=1024,
    check_embedding_ctx_length=False
)

Tips：在检索结果中，id 为 hybrid_1 和 hybrid_2 一样的原因是因为 HotpotQA 在「不同问题的不同段落间」或「同一个问题下的不同的段落间」可能会提供相同的内容，由于对段落的切篇方式相同，会造成内容完全一致的现象

具体到 hybrid_1 和 hybrid_2 来说，两者相同的原因是由于：「不同问题间出现了相同的段落」。由于两者来自不同的问题，各自的主键 pk 是不同的，您可以在最下面代码的检索结果中验证这一说法（「hybrid_6 和 hybrid_7」、「dense_3 和 dense_4」的重复也是类似的情况）

In [ ]:
# 定义要存储的文档
from langchain_core.documents import Document

docs = [
    Document(
        page_content='ICI House (now Orica House) is a 19-storey office building in East Melbourne, Victoria, Australia. Begun in 1955, it was the tallest building in Australia upon completion in 1958, breaking Melbourne\'s long standing 132ft height limit, and was the first International Style skyscraper in the country. It symbolised progress, modernity, efficiency and corporate power in postwar Melbourne, and heralded the construction of the high-rise office buildings',
        metadata={
            'id': 'dense_1',
            'score': 0.7522499561309814
        }
    ),
    Document(
        page_content=', then a record among magazines published by Time, Inc., SI\'s parent company.',
        metadata={
            'id': 'hybrid_1',
            'score': 0.9679315090179443
        }
    ),
    Document(
        page_content='ICI Fibres developed the Crimplene fibre. It is a thick, polyester yarn used to make a fabric of the same name. The resulting cloth is heavy, wrinkle-resistant and retains its shape well. Britain\'s defunct ICI Laboratory developed the fibre in the early 1950s and named it after the Crimple Valley in which the company was situated.',
        metadata={
            'id': 'dense_2',
            'score': 0.8594906330108643
        }
    ),
    Document(
        page_content=', then a record among magazines published by Time, Inc., SI\'s parent company.',
        metadata={
            'id': 'hybrid_2',
            'score': 0.9679315090179443
        }
    ),
    Document(
        page_content=', then a record among magazines published by Time, Inc., SI\'s parent company.',
        metadata={
            'id': 'dense_3',
            'score': 0.9679315090179443
        }
    ),
    Document(
        page_content='Villard, also known as Villard Books, is a publishing imprint of Random House, one of the largest publishing companies in the world. It was founded in 1983. Villard began as an independent imprint of Random House and is currently a sub-imprint of Ballantine Books, itself an imprint of Random House. It was named after a Stanford White brownstone mansion on Madison Avenue that was the home of Random House for twenty years.',
        metadata={
            'id': 'hybrid_3',
            'score': 0.989403247833252
        }
    ),
    Document(
        page_content=', then a record among magazines published by Time, Inc., SI\'s parent company.',
        metadata={
            'id': 'dense_4',
            'score': 0.9679315090179443
        }
    ),
    Document(
        page_content=', Illinois, and they have an annual catalog circulation exceeding 50 million. The company is employee owned and considered to be a renowned purveyor of gadgetry and elegant gifts. Every item that is sold comes with the Hammacher Schlemmer Lifetime Guarantee ("We make an unconditional and unwavering promise: Our merchandise is guaranteed for life")',
        metadata={
            'id': 'hybrid_4',
            'score': 0.9983643293380737
        }
    ),
    Document(
        page_content='Izze (pronounced iz-ee) is the brand name of a line of carbonated juice drinks produced by the IZZE Beverage Company in Boulder, Colorado, which is owned by PepsiCo. The drinks consist of 70% fruit juice from concentrate, and 30% seltzer water. Izze is an all-natural, no-preservatives-added fruit soda.',
        metadata={
            'id': 'dense_5',
            'score': 0.9726308584213257
        }
    ),
    Document(
        page_content='Dodge Correctional Institution, (DCI), is an adult male maximum-security correctional facility operated by the Wisconsin Department of Corrections Division of Adult Institutions in Waupun, Wisconsin. The facility was converted from the Central State Hospital for the Criminally Insane to an adult correctional facility in 1977 at a cost of $2.47 million of general obligation bonds',
        metadata={
            'id': 'hybrid_5',
            'score': 1.0290902853012085
        }
    ),
    Document(
        page_content='The Tongil Industries Company Co., Ltd., (in short the “TIC”), is a South Korean heavy industry company headquartered in Changwon City, South Korea. TIC was founded in July, 1988 originally as the Jin Heung Machinery Co., Ltd. As of 2011, it comprises 4 business divisions; Machine tools, Ball Screws, Automobile Components and Heat Treatment. The Tongil Industries is a subsidiary of the TONGIL Group, a South Korean business conglomerate (chaebol) managed by Kook Jin “Justin” Moon',
        metadata={
            'id': 'dense_6',
            'score': 0.9825005531311035
        }
    ),
    Document(
        page_content=', yet the crash created "Watch Dog" like HAMP TEAM, IRS, COMPTROLLER< Federal Trade Commission Consumer Protection Bureau, FBI, CIA, Local Police Department, ICE ( The FBI online Computer crime division receives and investigates computer crimes that record keeping staff from title companies, lending institutional staff',
        metadata={
            'id': 'hybrid_6',
            'score': 1.0295854806900024
        }
    ),
    Document(
        page_content=', to Morse International. Morse filed for bankruptcy and Diving Equipment and Supply Company (DESCO) acquired its assets in 2016. DESCO reverted to the name A J Morse & Son and Morse products will be marketed under that name. DESCO\'s business plan is to bring back the quality and products associated with the earlier name. DESCO has on re-introduced the breast plate feed (air being fed into the breast plate rather than the bonnet)helmet design from the early 1900s as its first offering. They also',
        metadata={
            'id': 'dense_7',
            'score': 0.9825825691223145
        }
    ),
    Document(
        page_content=', yet the crash created "Watch Dog" like HAMP TEAM, IRS, COMPTROLLER< Federal Trade Commission Consumer Protection Bureau, FBI, CIA, Local Police Department, ICE ( The FBI online Computer crime division receives and investigates computer crimes that record keeping staff from title companies, lending institutional staff',
        metadata={
            'id': 'hybrid_7',
            'score': 1.0295854806900024
        }
    ),
    Document(
        page_content='Desilu Productions ( ) was an American production company founded and co-owned by husband and wife Desi Arnaz and Lucille Ball, best known for shows such as "I Love Lucy", "", and "The Untouchables". Until 1962, Desilu was the second-largest independent television production company in the U.S. behind MCA\'s Revue Productions until MCA bought Universal Pictures',
        metadata={
            'id': 'dense_8',
            'score': 0.9858689904212952
        }
    ),
    Document(
        page_content='Coty, Inc., is a North American beauty products manufacturer based in New York founded in Paris, France, by François Coty in 1904. Its main products are fragrances, colour cosmetics and skin and body care products. It is known for its cooperation with designers and celebrities for the creation of fragrances. Its biggest brands, or "power brands" as it calls them, are: Calvin Klein (fragrance and cosmetics), Chloe (fragrance), Davidoff (fragrance), y (fragrance), Marc Jacobs (fragrance)',
        metadata={
            'id': 'hybrid_8',
            'score': 1.0300822257995605
        }
    ),
    Document(
        page_content='Villard, also known as Villard Books, is a publishing imprint of Random House, one of the largest publishing companies in the world. It was founded in 1983. Villard began as an independent imprint of Random House and is currently a sub-imprint of Ballantine Books, itself an imprint of Random House. It was named after a Stanford White brownstone mansion on Madison Avenue that was the home of Random House for twenty years.',
        metadata={
            'id': 'dense_9',
            'score': 0.989403247833252
        }
    ),
    Document(
        page_content=', clothing, and fragrances being released.',
        metadata={
            'id': 'hybrid_9',
            'score': 1.0305763483047485
        }
    ),
    Document(
        page_content=', Illinois, and they have an annual catalog circulation exceeding 50 million. The company is employee owned and considered to be a renowned purveyor of gadgetry and elegant gifts. Every item that is sold comes with the Hammacher Schlemmer Lifetime Guarantee ("We make an unconditional and unwavering promise: Our merchandise is guaranteed for life").',
        metadata={
            'id': 'dense_10',
            'score': 0.9983643293380737
        }
    ),
    Document(
        page_content=', as well as catering items such as sandwich trays and boxed lunches. The chain is also known for its McAlister\'s Famous Sweet Tea, which is available by the glass or by the gallon.',
        metadata={
            'id': 'hybrid_10',
            'score': 1.0356755256652832
        }
    )

]

# 为其分配 id 
ids = [str(i + 1) for i in range(len(docs))]

In [ ]:
# 使用 langchain_milvus 定义「仅包含密集嵌入的向量数据库」和「包含稀疏嵌入和密集嵌入的向量数据库」
from langchain_milvus import Milvus, BM25BuiltInFunction

# 仅包含密集嵌入的向量数据库
dense_vs = Milvus(
    collection_name='dense_only',
    embedding_function=embed_model,
    vector_field='dense'
)
# 向其中添加实体
dense_vs.add_documents(documents=docs[:10], ids=ids[:10])  # 嵌入模型一次无法接受太多文档7])  # 嵌入模型一次无法接受太多文档
dense_vs.add_documents(documents=docs[10:], ids=ids[10:])


# 包含稀疏嵌入和密集嵌入的向量数据库
hybrid_vs = Milvus(
    collection_name='dense_and_sparse',
    embedding_function=embed_model,
    builtin_function=BM25BuiltInFunction(output_field_names='sparse'),
    vector_field=['dense', 'sparse']
)
# 向其中添加实体
hybrid_vs.add_documents(documents=docs[:10], ids=ids[:10])
hybrid_vs.add_documents(documents=docs[10:], ids=ids[10:])

['11', '12', '13', '14', '15', '16', '17', '18', '19', '20']

In [ ]:
# 检索
query = 'ICI House is now named after the company that provides what type of item?'

dense_res = dense_vs.similarity_search_with_score(
    query=query,
    k=10
)

hybrid_output_fields = hybrid_vs.fields[:]
hybrid_output_fields.remove('sparse')

raw_hybrid_res = hybrid_vs.client.search(
    collection_name='dense_and_sparse',
    data=[embed_model.embed_query(query)],
    anns_field='dense',
    limit=10,
    search_params={
        'metric_type': 'L2',
        'params': {}
    },
    output_fields=hybrid_output_fields
)
hybrid_res = hybrid_vs._parse_documents_from_search_results(raw_hybrid_res)

In [ ]:
import pprint

pprint.pprint(dense_res)

[(Document(metadata={'pk': '1', 'id': 'dense_1', 'score': 0.7522499561309814}, page_content="ICI House (now Orica House) is a 19-storey office building in East Melbourne, Victoria, Australia. Begun in 1955, it was the tallest building in Australia upon completion in 1958, breaking Melbourne's long standing 132ft height limit, and was the first International Style skyscraper in the country. It symbolised progress, modernity, efficiency and corporate power in postwar Melbourne, and heralded the construction of the high-rise office buildings"),
  0.7522499561309814),
 (Document(metadata={'pk': '3', 'id': 'dense_2', 'score': 0.8594906330108643}, page_content="ICI Fibres developed the Crimplene fibre. It is a thick, polyester yarn used to make a fabric of the same name. The resulting cloth is heavy, wrinkle-resistant and retains its shape well. Britain's defunct ICI Laboratory developed the fibre in the early 1950s and named it after the Crimple Valley in which the company was situated."),


In [ ]:
pprint.pprint(hybrid_res)

[(Document(metadata={'score': 0.7522499561309814, 'pk': '1', 'id': 'dense_1'}, page_content="ICI House (now Orica House) is a 19-storey office building in East Melbourne, Victoria, Australia. Begun in 1955, it was the tallest building in Australia upon completion in 1958, breaking Melbourne's long standing 132ft height limit, and was the first International Style skyscraper in the country. It symbolised progress, modernity, efficiency and corporate power in postwar Melbourne, and heralded the construction of the high-rise office buildings"),
  0.7522499561309814),
 (Document(metadata={'score': 0.8594906330108643, 'pk': '3', 'id': 'dense_2'}, page_content="ICI Fibres developed the Crimplene fibre. It is a thick, polyester yarn used to make a fabric of the same name. The resulting cloth is heavy, wrinkle-resistant and retains its shape well. Britain's defunct ICI Laboratory developed the fibre in the early 1950s and named it after the Crimple Valley in which the company was situated."),


这次两个向量数据库的密集检索结果是一致的，并没有复现出之前的问题

In [ ]:
# 在上面的 document 中 id 为 hybrid_1、hubrid_2 的内容完全一样的原因是：
#  HotpotQA 在「不同问题的不同段落间」或「同一个问题下的不同的段落间」可能会提供相同的内容，由于对段落的切篇方式相同，会造成内容完全一致的现象
# 具体到 hybrid_1 和 hybrid_2 来说，两者相同的原因是由于：「不同问题间出现了相同的段落」
# 下面是验证两者是 来自不同的问题的代码

from langchain_milvus import Milvus, BM25BuiltInFunction

# 加载两个嵌入类型不同的向量数据库
dense_500 = Milvus(
    collection_name='dense_hotpotqa_500',
    embedding_function=embed_model
)

hybrid_500 = Milvus(
    collection_name='hybrid_hotpotqa_500',
    embedding_function=embed_model,
    builtin_function=BM25BuiltInFunction(output_field_names='sparse'),
    vector_field=['dense', 'sparse']
)

target = ', then a record among magazines published by Time, Inc., SI\'s parent company.'
dense_res = dense_500.search_by_metadata(
    expr=f'text=="{target}"',
    fields=["*"]
)
hybrid_res = hybrid_500.search_by_metadata(
    expr=f'text=="{target}"',
    fields=["*"]
)

print('dense res comes from: ')
for d_r in dense_res:
    print(f'hotpotqa_id: {d_r.metadata['hotpotqa_id']}')
    print(f'question: {d_r.metadata["question"]}\n')

print('hybrid res comes from: ')
for h_r in hybrid_res:
    print(f'hotpotqa_id: {h_r.metadata["hotpotqa_id"]}')
    print(f'question: {h_r.metadata["question"]}\n')

dense res comes from: 
hotpotqa_id: 5a81b43d5542995ce29dcc39
question: John Stoltenberg is the managing editor of the magazine that focuses on what?

hotpotqa_id: 5a8198565542995ce29dcc13
question: Are Ranger Rick and Tennis both sports magazines?

hybrid res comes from: 
hotpotqa_id: 5a8198565542995ce29dcc13
question: Are Ranger Rick and Tennis both sports magazines?

hotpotqa_id: 5a81b43d5542995ce29dcc39
question: John Stoltenberg is the managing editor of the magazine that focuses on what?

